In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.metrics import mean_absolute_error

In [3]:
df = pd.read_csv('data/train.csv')
df.tail()

,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,coco,el_price,consumption
8587,2022-08-24 19:00:00+03:00,27.1,17.0,54.0,0.0,NaN,180.0,4.0,11.1,1020.0,2.0,0.53494,0.678
8588,2022-08-24 20:00:00+03:00,25.1,17.1,61.0,0.0,NaN,220.0,6.0,11.1,1021.0,1.0,0.49990,0.457
8589,2022-08-24 21:00:00+03:00,24.9,18.1,66.0,0.0,NaN,150.0,4.0,9.3,1020.9,4.0,0.43149,0.500
8590,2022-08-24 22:00:00+03:00,22.1,17.0,73.0,0.0,NaN,160.0,6.0,9.3,1021.0,2.0,0.55203,2.321
8591,2022-08-24 23:00:00+03:00,22.1,18.1,78.0,0.0,NaN,310.0,15.0,9.3,1021.0,2.0,0.43108,0.678


In [4]:
y_true = df.iloc[df.shape[0]-24*7:][['time', 'consumption']]
y_true

,time,consumption
8424,2022-08-18 00:00:00+03:00,0.245
8425,2022-08-18 01:00:00+03:00,0.178
8426,2022-08-18 02:00:00+03:00,0.203
8427,2022-08-18 03:00:00+03:00,0.215
8428,2022-08-18 04:00:00+03:00,1.043
...,...,...
8587,2022-08-24 19:00:00+03:00,0.678
8588,2022-08-24 20:00:00+03:00,0.457
8589,2022-08-24 21:00:00+03:00,0.500
8590,2022-08-24 22:00:00+03:00,2.321


XGBRegressor

In [5]:
import pandas as pd
from datetime import datetime as dt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [6]:
def generate_cyclical_features(df, col_name, period, start_num=0):
    kwargs = {
        f'sin_{col_name}': lambda x: np.sin(2 * np.pi * (df[col_name] - start_num) / period),
        f'cos_{col_name}': lambda x: np.cos(2 * np.pi * (df[col_name] - start_num) / period)
    }
    return df.assign(**kwargs)

In [7]:
df_XGB = df[['time', 'temp', 'el_price', 'coco', 'consumption']].dropna()

df_XGB['hour'] = df_XGB['time'].apply(lambda x: dt.strptime(str(x), "%Y-%m-%d %H:%M:%S%z").hour)
df_XGB['month'] = df_XGB['time'].apply(lambda x: dt.strptime(str(x), "%Y-%m-%d %H:%M:%S%z").month)

df_XGB = generate_cyclical_features(df_XGB, 'hour', 24, 0)
df_XGB = generate_cyclical_features(df_XGB, 'month', 12, 1)

Test data

In [37]:
df_kaggle_XGB = pd.read_csv('data/test.csv')

In [28]:
df_kaggle_XGB = df_kaggle_XGB[['time', 'temp', 'el_price', 'coco']].dropna()

df_kaggle_XGB['hour'] = df_kaggle_XGB['time'].apply(lambda x: dt.strptime(str(x), "%Y-%m-%d %H:%M:%S%z").hour)
df_kaggle_XGB['month'] = df_kaggle_XGB['time'].apply(lambda x: dt.strptime(str(x), "%Y-%m-%d %H:%M:%S%z").month)

df_kaggle_XGB = generate_cyclical_features(df_kaggle_XGB, 'hour', 24, 0)
df_kaggle_XGB = generate_cyclical_features(df_kaggle_XGB, 'month', 12, 1)

In [29]:
features = ['temp', 'el_price', 'coco', 'cos_hour', 'cos_month']
df_X_kaggle_XGB = df_kaggle_XGB[features]

In [21]:
print(df_XGB.shape)
test_days = 0
train_days = 14 * 24

test_df_XGB = df_XGB.iloc[df_XGB.shape[0]-test_days:] 
train_df_XGB = df_XGB.iloc[df_XGB.shape[0]-(test_days+train_days):df_XGB.shape[0]-test_days]
print(test_df_XGB.shape)
print(train_df_XGB.shape)

(8394, 11)
(0, 11)
(336, 11)


In [30]:
features = ['temp', 'el_price', 'coco', 'cos_hour', 'cos_month']

train_X_XGB = train_df_XGB[features]
train_y_XGB = train_df_XGB['consumption']

test_X_XGB = test_df_XGB[features]
test_y_XGB = test_df_XGB['consumption'] 

In [31]:
params = {
    "n_estimators": 200,
    "max_depth": 3,
    "min_samples_split": 2,
    "learning_rate": 0.02,
    "loss": "squared_error",
}
model = GradientBoostingRegressor(**params)
model.fit(train_X_XGB, train_y_XGB)

GradientBoostingRegressor(learning_rate=0.02, n_estimators=200)

In [33]:
pred = model.predict(df_X_kaggle_XGB)

In [38]:
submissions = pd.DataFrame({'time': df_kaggle_XGB['time'], 'consumption': pred})
submissions.to_csv('data/submissions_xgb.csv', index=False)

In [14]:
LR = pd.read_csv('data/best_LR_w_2_3_v2.csv')
LR

,time,consumption
0,2022-08-18 00:00:00+03:00,0.406242
1,2022-08-18 01:00:00+03:00,0.392767
2,2022-08-18 02:00:00+03:00,0.407683
3,2022-08-18 03:00:00+03:00,0.355203
4,2022-08-18 04:00:00+03:00,0.406936
...,...,...
163,2022-08-24 19:00:00+03:00,0.926008
164,2022-08-24 20:00:00+03:00,0.875579
165,2022-08-24 21:00:00+03:00,0.890830
166,2022-08-24 22:00:00+03:00,0.818715


In [18]:
y_true.reset_index(inplace=True)
y_true

,level_0,index,time,consumption
0,0,8424,2022-08-18 00:00:00+03:00,0.245
1,1,8425,2022-08-18 01:00:00+03:00,0.178
2,2,8426,2022-08-18 02:00:00+03:00,0.203
3,3,8427,2022-08-18 03:00:00+03:00,0.215
4,4,8428,2022-08-18 04:00:00+03:00,1.043
...,...,...,...,...
163,163,8587,2022-08-24 19:00:00+03:00,0.678
164,164,8588,2022-08-24 20:00:00+03:00,0.457
165,165,8589,2022-08-24 21:00:00+03:00,0.500
166,166,8590,2022-08-24 22:00:00+03:00,2.321


In [20]:
mean_absolute_error(y_true[['consumption']], LR[['consumption']])

0.36508137344936176